In [1]:
import sys
sys.path.insert(0, '../../../utils/')
import utils_regressor
import utils_date
import utils

from tqdm import tqdm


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [2]:
# Variables data shaping
features_time_step = ['2-end_event', '3-end_event', '4-end_event', '5-end_event',
               '6-end_event', '8-end_event', '9-end_event', '10-end_event',
               '11-end_event', '12-end_event', '13-end_event', '14-end_event',
               '15-end_event', '16-end_event', '23-end_event', '24-end_event',
               '30-end_event', '31-end_event', '32-end_event', '34-end_event',
               '35-end_event', '40-end_event', '43-end_event', '45-end_event',
               '47-end_event', '61-end_event', '68-end_event']

features_day = ['Day_en', 'Mois','vac_noel_quebec', 'day_off_quebec', '24DEC', '31DEC',
                    'renov_beaubien', 'vac_udem1', 'vac_udem2',]

features_todummy = ['Day_en', 'Mois']

time_series = ['11', '32', '34', '15', '44', '65', '31', '33', '35', '47', '13',
       '14', '1', '9', '5', '18', '36', '24', '68', '43', '8', '64', '10',
       '55', '3', '49', '51', '2', '19', '56', '7', '6', '4', '48', '66',
       '25', '23', '28', '39', '54', '60', '27', '20', '46', '12', '21',
       '62', '52', '41', '50', '30', '16', '37', '40', '26', '67', '57',
       '61', '42', '45', '38', '29', '58', '63', '22', '59', '53', '17']


start_datetime = '2015-01-01 00:00:00'
end_datetime = '2017-12-31 23:45:00'

# Variables optimization
scaler_choice_X = None
scaler_choice_y = None

start_datetime_optimization = '2015-01-01 00:00:00'
end_datetime_optimization = '2016-12-31 23:45:00'

param_kfold={
    'n_splits': 5,
    'shuffle': True,
    'random_state': 1}

param_grid={
    'n_estimators': [100, 150, 200],
    'max_features': ['auto',None],
    'max_depth': [None],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1,5,10],
    'n_jobs': [6],
    'criterion': ['mse']}

param_kfold={
    'n_splits': 2,
    'shuffle': True,
    'random_state': 1}

#rf
param_grid={
    'n_estimators': [10, 20],
    'max_features': [None],
    'max_depth': [None],
    'min_samples_split': [5],
    'min_samples_leaf': [10],
    'n_jobs': [6],
    'criterion': ['mse']}

#lr
param_grid={
    'fit_intercept':[True],
    'normalize':[False, True],
    'n_jobs': [1]}

#gp
param_grid={
    'kernel':  [None],
    'alpha':  [0.05, 0.1, 0.2, 0.3, 0.5],
    'optimizer':  ['fmin_l_bfgs_b'],
    'n_restarts_optimizer':  [0],
    'normalize_y':  [False, True],
    'copy_X_train':  [True],
    'random_state':  [None]
}


# Variables global
index = 'Datetime'
start_time = '00:00:00'
end_time = '23:45:00'
time_step_second=15*60

# Variables model
model_name = 'test'
estimator_choice = 'lr'

# Variables path
path_save = '/home/toque/data2/forecast/model/long-term/scikit-regressor/'

# Data files
observation_data_path = ['/home/toque/data2/montreal/stm/data/valid_metro_15min_2015_2016_2017_sumpass_nodayfree.csv']
features_data_path = ['/home/toque/data2/montreal/events/data/clean/events_2015_2018_end_event_stopid.csv',
#                      '/home/toque/data2/montreal/events/data/clean/events_2015_2018_start_event_stopid.csv',
#                      '/home/toque/data2/montreal/events/data/clean/events_2015_2018_period_event_stopid.csv',
#                      '/home/toque/data2/montreal/events/data/clean/events_2015_2018_end_event_stopid_category.csv',
#                      '/home/toque/data2/montreal/events/data/clean/events_2015_2018_start_event_stopid_category.csv',
#                      '/home/toque/data2/montreal/events/data/clean/events_2015_2018_period_event_stopid_category.csv',
                      '/home/toque/data2/weather/predicted_weather/predicted_weather_2015_2017_included_perday_pm.csv',
                      '/home/toque/data2/date/2013-01-01-2019-01-01_new.csv',
                      '/home/toque/data2/montreal/stm/data/trend/trend_r_stl_2015-01-01_2018-01-01_freq30.csv']


In [ ]:
parser = argparse.ArgumentParser(description='Parameters of script fit')
parser.add_argument('--config', type=str, help='Yaml file containing the configuration of the model')
parser.add_argument('--force', type=bool, help='Boolean: If True force all the saving even if file already exist, if False ask to the user',default=False)
args = parser.parse_args()
config_file = args.config

with open(config_file, 'r') as stream:
    try:
        config = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
        
        
        
force_save = args.force

features_time_step = config['features_time_step']
features_day = config['features_day']
features_todummy = config['features_todummy']
time_series = config['time_series']
start_datetime = config['start_datetime']
end_datetime = config['end_datetime']
scaler_choice_X = config['scaler_choice_X']
scaler_choice_y = config['scaler_choice_y']
start_datetime_optimization = config['start_datetime_optimization']
end_datetime_optimization = config['end_datetime_optimization']
param_kfold = config['param_kfold']
param_grid = config['param_grid']
index = config['index']
start_time = config['start_time']
end_time = config['end_time']
time_step_second = config['time_step_second']
model_name = config['model_name']
estimator_choice = config['estimator_choice']
path_save = config['path_save']
observation_data_path = config['observation_data_path']
features_data_path = config['features_data_path']


In [3]:
path_optimization = path_save+'optimize/{}/'.format(model_name)
path_prediction = path_save+'prediction/{}/'.format(model_name)
path_fit = path_save+'fit/{}/'.format(model_name)


model_infos = {}
model_infos['model_name'] = model_name
model_infos['estimator_choice'] = estimator_choice

model_infos['index'] = index
model_infos['start_time'] = start_time
model_infos['end_time'] = end_time
model_infos['time_step_second'] = time_step_second

model_infos['path_save'] = path_save
model_infos['path_optimization'] = path_optimization
model_infos['path_fit'] = path_fit
model_infos['path_prediction'] = path_prediction

model_infos['observation_data_path'] = observation_data_path
model_infos['features_data_path'] = features_data_path

model_infos['param_grid'] = param_grid
model_infos['param_kfold'] = param_kfold

model_infos['features_time_step'] = features_time_step
model_infos['features_day'] = features_day
model_infos['features_dummy'] = features_dummy
model_infos['time_series'] = time_series
model_infos['start_datetime'] = start_datetime
model_infos['end_datetime'] = end_datetime
model_infos['scaler_choice_X'] = scaler_choice_X
model_infos['scaler_choice_y'] = scaler_choice_y
model_infos['start_datetime_optimization'] = start_datetime_optimization
model_infos['end_datetime_optimization'] = end_datetime_optimization


df_obs = utils.read_csv_list(observation_data_path).set_index(index).loc[start_datetime:end_datetime].reset_index()
df_fea = utils.read_csv_list(features_data_path).set_index(index).loc[start_datetime:end_datetime].reset_index()

In [4]:
# Data shaping

dfXy, features_time_step_name = utils.create_dfXy_long_term0(df_obs, df_fea, time_series,
                                                             features_time_step, features_day,
                                                             features_todummy=features_todummy,
                                                             index=index, start_time=start_time,
                                                             end_time=end_time, time_step_second=time_step_second)

X, y_list, X_names, days = utils.create_xy_dataset_long_term0(dfXy, time_series, features_time_step_name=features_time_step_name,
                                                  index=index, start_time=start_time, end_time=end_time, reduce=True)

#dfX = pd.DataFrame(data = np.concatenate([np.array(days).reshape(1,1093),X.T]).T, columns=['Datetime']+X_names.tolist())


Days loop: 100%|██████████| 1093/1093 [00:09<00:00, 111.10it/s]


In [5]:
# Split train - (test)

df_train_datetime = dfXy[[index]].copy()
df_train_datetime[index] = [i[:10]+' '+start_time for i in df_train_datetime[index]]
df_train_datetime = df_train_datetime.drop_duplicates()

index_train = len(df_train_datetime.set_index(index).loc[start_datetime_optimization:end_datetime_optimization])

X_train, y_list_train, days_train = X[:index_train], y_list[:,:index_train], days[:index_train]
#X_test, y_list_test, days_test = X[index_train:], y_list[:,index_train:], days[index_train:]

# Scaler
# TODO

In [6]:
# Optimize

estimator = utils_regressor.get_estimator(estimator_choice)
   
grid_search_time_series = {}
grid_search_time_series = utils_regressor.optimize_multioutput_regressor_multiseries_model(X_train, y_list_train[:5],
                                                                           param_grid, param_kfold,
                                                                           estimator, verbose=0)

utils.save_pickle_safe(path_optimization+'grid_search_time_series.pkl', grid_search_time_series, force_save=force_save)
utils.save_pickle_safe(path_optimization+'model_infos.pkl', model_infos, force_save=force_save)

Optimization: loop over time-series: 100%|██████████| 5/5 [00:07<00:00,  1.49s/it]


In [7]:
# Fit

# Create manually the best_params
#dict[int]['best_params_']
#best_params = {'alpha': 0.2,'copy_X_train': True, 'kernel': None, 'n_restarts_optimizer': 0, 'normalize_y': True,
#               'optimizer': 'fmin_l_bfgs_b','random_state': None}
#best_params_time_series = {i : {'best_params':best_params} for i in np.arange(len(time_series))}

grid_search_time_series = utils.load_pickle(path_optimization+'grid_search_time_series.pkl')
best_params_time_series = grid_search_time_series

estimator_time_series = utils_regressor.fit_multioutput_regressor_multiseries_model(X_train, y_list_train[:5], estimator_choice, best_params_time_series)   

Fit: loop over time-series: 100%|██████████| 5/5 [00:00<00:00, 10.45it/s]


In [14]:
# Predict

pred = utils_regressor.predict_multioutput_regressor_multiseries_model(X, estimator_time_series)
df = utils.pred_day_array_to_df(pred, time_series[:5], days, time_step_second=15*60, index='Datetime')

utils.df_to_csv_safe(path_prediction+start_datetime[:10]+'_'+end_datetime[:10]+'.csv', df, force_save=force_save)

Predict: loop over time-series: 100%|██████████| 5/5 [00:00<00:00,  7.63it/s]


In [83]:
aa = pd.read_csv('/home/toque/data2/montreal/events/data/clean/events_2015_2018_end_event_stopid.csv')
bb = pd.read_csv('/home/toque/data2/montreal/events/data/clean/events_2015_2018_start_event_stopid.csv')
cc = pd.read_csv('/home/toque/data2/montreal/events/data/clean/events_2015_2018_period_event_stopid.csv')
dd = pd.read_csv('/home/toque/data2/montreal/events/data/clean/events_2015_2018_end_event_stopid_category.csv')
ee = pd.read_csv('/home/toque/data2/montreal/events/data/clean/events_2015_2018_start_event_stopid_category.csv')
ff = pd.read_csv('/home/toque/data2/montreal/events/data/clean/events_2015_2018_period_event_stopid_category.csv')

In [85]:
np.array(bb.columns.tolist()[1:] + aa.columns.tolist()[1:] + cc.columns.tolist()[1:] +  dd.columns.tolist()[1:] + ff.columns.tolist()[1:] + ee.columns.tolist()[1:] )

array(['1-start_event', '2-start_event', '3-start_event', '4-start_event',
       '5-start_event', '6-start_event', '8-start_event', '9-start_event',
       '10-start_event', '11-start_event', '12-start_event',
       '13-start_event', '14-start_event', '15-start_event',
       '16-start_event', '23-start_event', '24-start_event',
       '29-start_event', '30-start_event', '31-start_event',
       '32-start_event', '34-start_event', '35-start_event',
       '40-start_event', '43-start_event', '45-start_event',
       '47-start_event', '61-start_event', '68-start_event', '2-end_event',
       '3-end_event', '4-end_event', '5-end_event', '6-end_event',
       '8-end_event', '9-end_event', '10-end_event', '11-end_event',
       '12-end_event', '13-end_event', '14-end_event', '15-end_event',
       '16-end_event', '23-end_event', '24-end_event', '30-end_event',
       '31-end_event', '32-end_event', '34-end_event', '35-end_event',
       '40-end_event', '43-end_event', '45-end_event', '47-

In [97]:
df


,0,1,2
0,1,2,3
1,1,1,1
2,133,1,33
